<a href="https://colab.research.google.com/github/lauren-esser/capstone-predicting-the-stock-market/blob/master/Capstone11_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Final Project Submission:

**Student Name:** Lauren Esser

**Student Pace:** Full Time

**Scheduled Project Review Date/Time:**  

**Instructor Name:** James Irving

**Blog Post URLS:**  https://lauren-esser.github.io/how_to_use_newsapi and 

#Abstract

An Abstract section that briefly explains your problem, your methodology, and your findings, and business recommendations as a result of your findings. This section should be 1-2 paragraphs long.


#Import

In [ ]:
import numpy as np
np.random.seed(1919)

In [ ]:
import tensorflow as tf
tf.random.set_seed(1919)

In [ ]:
#import other libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns
import os, glob
from statsmodels.tsa.stattools import adfuller


In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

In [ ]:
%cd ~
%cd ..

In [ ]:
!ls "/gdrive/My Drive/Colab Notebooks/DataSets"

In [ ]:
print(os.path.abspath(os.curdir))
os.listdir()

#Obtain

Obtain Stocks Dataset:
1. Go to http://www.kibot.com/free_historical_data.aspx
2. Scroll to bottom of page and download "Tick with bid/ask data" under IVE(S&P 500 Value Index) 
3. Upload to Google drive
4. Follow below steps

In [ ]:
#import and assign txt file
source_folder = r'/gdrive/My Drive/Colab Notebooks/DataSets/'
target_folder = r'/content/'
file = glob.glob(source_folder+ 'IVE_tickbidask.txt', recursive = True)
file = file[0]
file

In [ ]:
#define and view txt file
stocks = pd.read_csv(file, header = None)
stocks.head()

In [ ]:
#rename column headers
stocks = stocks.rename(columns={0: 'Date', 1:'Time', 2: 'Price', 3:'Bid',
                                4: 'Ask', 5:'Size'})
stocks

#### News API: Can only access up to one month of news headlines for free

In [ ]:
#pip install newsapi-python

In [ ]:
# from newsapi import NewsApiClient

In [ ]:

# connect with API key
# newsapi = NewsApiClient(api_key = '')

In [ ]:
#identify news source options
# news_sources = newsapi.get_sources()
# news_sourcecs_list = []
# for sources in news_sources['sources']:
#   news_sourcecs_list.append(sources['name'])
#   print(sources['name'])

In [ ]:
# #test out newsapi.get_everything for specific date
# date = '2020-11-01'
# news_headlines_everything = newsapi.get_everything('&from = (date)', '&to = (date1)', language = 'en')
                                                
# news_titles_everything = []
# for article in news_headlines_everything['articles']:
#   news_titles_everything.append(article['title'])
#   print(article['title'])

In [ ]:
# #create empty list
# news_titles = []

# #create function
# def get_news_titles(date):
#   '''Used to get a list of 20 news titles from usatoday on date specified via newsapi.org.

#   Args: date = desired date

#   Returns: List of 20 news articles from the specified date with the date listed after

#   Example: get_news_titles('2020-11-01')'''
  
#   date = date
#   news_headlines_everything = newsapi.get_everything(from_param = (date), to = (date), language = 'en', domains = 'usatoday.com')
#   for article in news_headlines_everything['articles']:
#     news_titles.append([article['title'], date])
#     #print(article['title'])
#     #return news_titles

In [ ]:
# get_news_titles('2020-11-01')

In [ ]:
# news_titles

In [ ]:
# get_news_titles('2020-10-25')
# news_titles

In [ ]:
# len(news_titles)

In [ ]:
# # sunday_news = ['2020-01-05', '2020-01-12', '2020-01-19', '2020-01-26', '2020-02-02', 
#                '2020-02-09', '2020-02-16', '2020-02-23', '2020-03-01', '2020-03-08',
#                '2020-03-15', '2020-03-22', '2020-03-29', '2020-04-05', '2020-04-12',
#                '2020-04-19', '2020-04-26', '2020-05-03', '2020-05-10', '2020-05-17',
#                '2020-05-24', '2020-05-31', '2020-06-07', '2020-06-14', '2020-06-21',
#                '2020-06-28', '2020-07-05', '2020-07-12', '2020-07-19', '2020-07-26', 
#                '2020-08-02', '2020-08-09', '2020-08-16', '2020-08-23', '2020-08-30',
#                '2020-09-06', '2020-09-13', '2020-09-20', '2020-09-27', '2020-10-04', 
#                '2020-10-11', '2020-10-18', '2020-10-25', '2020-11-01']

In [ ]:
# for date in sunday_news:
#   new = get_news_titles(date)
#   news_titles.append(new)
# return news_titles

In [ ]:
#newsapi.get_top_headlines for today
# news_headlines = newsapi.get_top_headlines(language = 'en', country = 'us')
# news_titles = []
# for article in news_headlines['articles']:
#   news_titles.append(article['title'])
#   print(article['title'])

#### Obtain News Dataset

News Category Dataset from Kaggle:
1. Go to https://www.kaggle.com/rmisra/news-category-dataset
2. Download dataset to your computer
3. Upload dataset into your Google Drive folder
4. Follow steps below

In [ ]:
!ls "/gdrive/My Drive/Colab Notebooks/DataSets"

In [ ]:
print(os.path.abspath(os.curdir))
os.listdir()

In [ ]:
source_folder = r'/gdrive/My Drive/Colab Notebooks/DataSets/'
target_folder = r'/content/'
file = glob.glob(source_folder+ 'archive.zip', recursive = True)
file = file[0]
file

In [ ]:
#upzip data
zip_path = file
!cp '{zip_path}' .

!unzip -q archive.zip
!rm archive.zip

In [ ]:
#open file for viewing
news = pd.read_json('/gdrive/My Drive/Colab Notebooks/DataSets/News_Category_Dataset_v2.json', lines = True)

In [ ]:
news.head(20)

#Scrub

### Scrubbing Stocks

In [ ]:
#check nulls
stocks.isna().sum()

In [ ]:
stocks.info()

In [ ]:
#convert date to a datetime column
stocks['Date'] = pd.to_datetime(stocks['Date'])#, format = '%d/%m/%y')

In [ ]:
stocks.info()

In [ ]:
#set date to index
stocks.set_index('Date', inplace= True)

In [ ]:
stocks.index

In [ ]:
stocks.head()

In [ ]:
stocks.info()

### Scrubbing News

In [ ]:
news.head()

In [ ]:
news.info()

In [ ]:
news.isna().sum()

In [ ]:
news.nunique()

In [ ]:
#check diff categories
news.category.value_counts()

In [ ]:
#end date
news.date.max()

In [ ]:
#start date
news.date.min()

In [ ]:
news.set_index('date', inplace= True)

In [ ]:
news.head(20)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

In [ ]:
stopwords = stopwords.words('english')
stopwords += list(string.punctuation)

In [ ]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
news_token = []
for headline in news.headline:
  tokens = nltk.regexp_tokenize(headline, pattern)
  stopped_tokens = [w.lower() for w in tokens if w not in stopwords]
  news_token.append(stopped_tokens)

In [ ]:
news_token

In [ ]:
num_dims = len(news_token)
print("Number of Dimensions: {}".format(num_dims))

In [ ]:
vals_list = []

for i in news_token:
  vals_list.append(list(i.values()))

vals_list[0][:10]

In [ ]:
from nltk import FreqDist

In [ ]:
freqdist = FreqDist(news_token)
most_common = freqdist.most_common(200)
most_common

In [ ]:
news.head()

#Explore

### Stocks Exploration

In order to make sure we are looking at the same time frame for both stocks and news articles, I will slice and save the correct dates for stocks to correspond to the news articles: 2012-01-28 to 2018-05-26.


In [ ]:
stocks_news = stocks['2012-01-28': '2018-05-26']
stocks_news.head()

I want to look at weekly mean of stocks for comparison during my Exploration.

In [ ]:
#downsampling to weekly price if needed
price_weekly = stocks_news['Price'].resample('W')
weekly_mean = price_weekly.mean()
weekly_mean.head()

In [ ]:
weekly_mean.isna().sum()

###### Line Plot

In [ ]:
#line plot of stocks price over the years
fig, (ax, ax1) = plt.subplots(2, figsize = (15, 8))
ax.plot(stocks_news['Price'])
ax.set_title('Stock Price Through The Years')
ax.set_ylabel('Price')
ax.set_xlabel('Year')

#weekly lineplot 
ax1.plot(weekly_mean)
ax1.set_title('Weekly Mean of Stock Price Through The Years')
ax1.set_ylabel('Price')
ax1.set_xlabel('Year')

#label outer to see 2nd title
ax.label_outer()

---

###### Dot Plot

In [ ]:
#dot plot of stocks price over the years
fig, (ax, ax1) = plt.subplots(2, figsize = (15, 8))
ax.plot(stocks_news['Price'], '.')
ax.set_title('Stock Price Through The Years')
ax.set_ylabel('Price')
ax.set_xlabel('Year')

#weekly dot plot 
ax1.plot(weekly_mean, '.')
ax1.set_title('Weekly Mean of Stock Price Through The Years')
ax1.set_ylabel('Price')
ax1.set_xlabel('Year')

#label outer to see 2nd title
ax.label_outer()

---

###### Rolling Statistics

In [ ]:
roll_mean = stocks_news['Price'].rolling(window = 8, center = False).mean()
roll_std = stocks_news['Price'].rolling(window = 8, center = False).std()

In [ ]:
fig = plt.figure(figsize = (15, 8))
plt.plot(stocks_news['Price'], color = 'blue', label = 'Original')
plt.plot(roll_mean, color = 'green', label = 'Rolling Mean')
plt.plot(roll_std, color = 'black', label = 'Rolling Std')
plt.legend()
plt.title('Rolling Mean & Standard Deviation')
plt.show()

---

###### The Dickey-Fuller Test

In [ ]:
dftest = adfuller(weekly_mean)

dfoutput = pd.Series(dftest[0:4], index = ['Test Statistic', 'p-value', '#Lags Used',
                                           'Number of Observations Used'])
for key, value in dftest[4].items():
  dfoutput['Critical Values (%s)' %key] = value
print(dftest)

In [ ]:
print('Results of Dicky-Fuller test: \n')

print(dfoutput)

**Summary:**
- Not a stationary series
- Linear upward trend
- Big drop near the end of 2015.


---

###### Grouping and Visualizing Time Series

In [ ]:
stocks_news.head()

In [ ]:
stocks_news.index

James suggestion in class: kinda confused, may need to wait for one on one.

In [ ]:
#group values annually
year_groups = stocks_news.groupby(pd.Grouper(freq = 'A'))

In [ ]:
# #add year column for ease of use
# stocks_news['year'] = stocks_news.index.to_series().apply(lambda x: x.year)

# #check column
# stocks_news.head()

In [ ]:
# #create new dataframe to store yearly values
# stocks_annual = pd.DataFrame()

# for yr, group in year_groups:
#   stocks_annual[yr.year] = group.values.ravel()

# #plot yearly subplots
# stocks_annual.plot(figsize=(15, 8), subplots = True, legend = True);
# plt.show()

In [ ]:
# year_groups.keys()

In [ ]:
#plot all years on same graph
# stocks_annual.plot(figsize - (15,8), subplots = False, legend = True)
# plt.show()

###### Histograms

In [ ]:
#build histogram to check distribution
fig, ax = plt.subplots(figsize = (12, 5))
ax.hist(stocks_news['Price'])
ax.set_title('Stock Price')
plt.show()

###### Density Plot

In [ ]:
#build density plot to check distribution
stocks_news['Price'].plot(kind = 'kde', figsize = (12, 5));
ax.set_title('Density Plot of Stock Price')

###### Time Series Box and Whisker Plots by Year

In [ ]:
#build box and whiskers plot by year
#stocks_annual.boxplot(figsize = (12, 5));

###### Time Series Heat Map

In [ ]:
#year_matrix = stocks_annual.T
#plt.matshow(year_matrix, interpolation = None, aspect = 'auto');

#### Eliminating the trend

##### Square Root Transform

In [ ]:
data = pd.Series(np.sqrt(stocks_news['Price']))
fig = plt.figure(figsize = (15, 8))
plt.plot(data, color = 'blue')
plt.xlabel('Year')
plt.ylabel('sqrt(yearly sales')
plt.show()

##### Log Transformation

In [ ]:
data = pd.Series(np.log(stocks_news['Price']))
fig = plt.figure(figsize = (15, 8))
plt.plot(data, color = 'blue')
plt.xlabel('Year')
plt.ylabel('log(yearly sales')
plt.show()

##### Differencing


In [ ]:
data_diff = data.diff(periods = 1)
data_diff.head()

In [ ]:
fig = plt.figure(figsize=(11,7))
plt.plot(data_diff, color='blue',label='Stock Price - rolling mean')
plt.legend(loc='best')
plt.title('Differenced Stock Price series')
plt.show(block=False)

In [ ]:
data_diff = data.diff().dropna()
data_diff.head()

In [ ]:
# data_diff = data_diff.groupby(pd.Grouper(freq = 'A'))
# data_diff.head()

### News Exploration

In [ ]:
fig = plt.figure(figsize = (15, 8))
sns.displot(news.category)
plt.title('Headline Count of Each Genre');

In [ ]:
news['year'] = news.index.to_series().apply(lambda x: x.year)

In [ ]:
news.head()

In [ ]:
sns.displot(news.year)
plt.title('Headline Count per Year');

#Model

#### Stocks Basic Time Series Model

##### Random Walk Model
- no specified mean or variance
- a strong dependence over time

In [ ]:
#series with the specified dates
work = pd.date_range('2012-01-28', '2018-05-26', freq = 'D')

#white noise error 
error = np.random.normal(0, 10, len(work))

#random walk
def random_walk(start, error):
  Y_0 = start
  cum_error = np.cumsum(error)
  Y = cum_error + Y_0
  return Y

#value of share on first date
share_value = random_walk(61, error)

shares_series = pd.Series(share_value, index = work)

In [ ]:
ax = shares_series.plot(figsize = (15, 8))
ax.set_xlabel('Date')
ax.set_ylabel('Stock Price')
ax.set_title('Random Walk')
plt.show()

##### Random Walk with a Drift

In [ ]:
#series with the specified dates
work = pd.date_range('2012-01-28', '2018-05-26', freq = 'D')

#white noise error 
error = np.random.normal(0, 10, len(work))

#random walk
def random_walk(start, error):
  Y_0 = start
  #add drift of 8
  cum_error = np.cumsum(error + 8)
  Y = cum_error + Y_0
  return Y

#value of share on first date
share_value_drift = random_walk(61, error)

shares_series_diff = pd.Series(share_value_drift, index = work)

In [ ]:
ax = shares_series_diff.plot(figsize = (15, 8))
ax.set_xlabel('Date')
ax.set_ylabel('Stock Price')
ax.set_title('Random Walk with Drift')
plt.show()

##### Differencing in a Random Walk Model

In [ ]:
#no drift
shares_diff = shares_series.diff(periods = 1)

fig = plt.figure(figsize = (15, 8))
plt.plot(shares_diff)
plt.title('Differenced Shares Series')
plt.show()

In [ ]:
#drift
shares_drift_diff = shares_series_diff.diff(periods = 1)

fig = plt.figure(figsize = (15, 8))
plt.plot(shares_drift_diff)
plt.title('Differenced Shares Series')
plt.show()

###### ARMA Model 

In [ ]:
stocks_news.head()

In [ ]:
data_diff.head()

In [ ]:
# stocks_news2 = stocks_news.copy()

In [ ]:
# #plotting the ACF
# from statsmodels.graphics.tsaplots import plot_acf
# fig, ax = plt.subplots(figsize = (8, 3))
# plot_acf(data_diff, ax = ax, lags = 8)

In [ ]:
# #plotting PACF
# from statsmodels.graphics.tsaplots import plot_pacf
# fig, ax = plt.subplots(figsize= (8, 3))
# plot_pacf(data_diff, ax = ax, lags = 8);

In [ ]:
# #import arma
# from statsmodels.tsa.arima_model import ARMA

# #fit an ARMA model
# mod_arma = ARMA(data_diff, order = (HOW DO I SELECT ORDER? CLASS HAS (1,0))
# res_arma = mod_arma.fit()

# print(res_arma.summary())

After figuring out ARMA model then try diff orders to select the best. Select it based on AIC or. BIC as model selection criterion.

#### ARIMA Model

q - auto regressive part of the model. effect of past values into model. 
d - amount of differnecing as it identifies the number of lag values to subtract from the current observation
q - moving average part of the model which is used to set the error of the model as a linear combination of the error values observed at previous time points in the past.

In [ ]:
stocks_news2 = stocks_news.copy()
stocks_news2.head()

In [ ]:
# The 'MS' string groups the data in buckets by start of the month
stocks_news2 = stocks_news2['Price'].resample('MS').mean()

# The term bfill means that we use the value before filling in missing values
stocks_news2 = stocks_news2.fillna(stocks_news2.bfill())

# Plot the time series
stocks_news2.plot(figsize=(15, 6))
plt.show()
print(stocks_news2.head())

In [ ]:
import itertools

In [ ]:
#define p,d,q with value between 0 & 2
p = d = q = range(0, 2)

#generate diff combos of p, q, & d triplets
pdq = list(itertools.product(p, d, q))

#generate all diff combos of seasonal p, d, q, triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

Try to find the model with the lowest AIC score

In [ ]:
#initialize empty list
answer = []

#iterate through params
for comb in pdq:
  for combs in pdqs:
    try:
      mod = sm.tsa.statespace.ARIMAX(stocks_news2,
                                      order = comb,
                                      seasonal_order = combs,
                                      enforce_stationarity = False,
                                      enforce_invertibility = False)
      
      output = mod.fit()
      answer.append([comb, combs, output.aic])
      print('ARIMA{} x{}12: AIC Calculated = {}'.format(comb, combs, output.aic))
    except:
      continue

In [ ]:
# Find the parameters with minimal AIC value
ans_df = pd.DataFrame(answer, columns=['pdq', 'pdqs', 'aic'])
ans_df.loc[ans_df['aic'].idxmin()]

#Interpret

#Summary

#Future Work